In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import graphviz 
# might need to install -- need *both* python and system
# so use Anaconda (or pip) to install graphviz, and also https://graphviz.org/download/
# this was a non-trivial install on Mac (depends on Xcode command-line tools)
# `pip install graphviz` and `brew install graphviz`
import datetime as dt

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

In [2]:
bb = pd.read_csv("../../Data/1999_2021/aggragate_daily_data_2000to2020.csv", index_col = "date")['influent_flow'].loc['2007-01-01':'2017-12-31'].to_frame()

weather = pd.read_csv("../../Data/historic_weather_all_SP_daily.csv", index_col = "date")

Changing columns to (station, measurement)

In [3]:
### weather_tmp = weather.pivot(columns='station', values=['rain_mm', 'temp_max','temp_min'])



In [4]:
'''#really ugly. sorry.
weather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)
weather_7 = weather_tmp.shift(-7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)
weather_14 = weather_tmp.shift(-14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)
weather_30 = weather_tmp.shift(-30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)
'''

"#really ugly. sorry.\nweather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)\nweather_7 = weather_tmp.shift(-7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)\nweather_14 = weather_tmp.shift(-14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)\nweather_30 = weather_tmp.shift(-30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)\n"

In [5]:
rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)
rain_0 = rain.rename(columns={'rain_mm':'rain_mm_0'})
rain_7 = rain.shift(-7).rename(columns={'rain_mm':'rain_mm_7'})
rain_14 = rain.shift(-14).rename(columns={'rain_mm':'rain_mm_14'})
rain_30 = rain.shift(-30).rename(columns={'rain_mm':'rain_mm_30'})

rain_agg = bb.join(rain_0).join(rain_7).join(rain_14).join(rain_30)


/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [6]:
X_agg = rain_agg.drop('influent_flow', axis = 1)
y_agg = rain_agg.influent_flow
X_agg_train, X_agg_test, y_agg_train, y_agg_test = train_test_split(X_agg, y_agg, train_size = 0.8, random_state = 0)

In [7]:
rain_0 = rain_0.join(bb).dropna()
rain_7 = rain_7.join(bb).dropna()
rain_14 = rain_14.join(bb).dropna()
rain_30 = rain_30.join(bb).dropna()

/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


## This is all stations with lags of 7, 14, and 30. 

In [9]:
# All stations, all lags predicting energy generated at B. Bonita

random_forest_rain_agg = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_agg.fit(X_agg_train, y_agg_train)

pred_agg = random_forest_rain_agg.predict(X_agg_test)

Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_agg_test, pred_agg))

random forest MAE: 184.89466679104476


In [11]:
Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

In [13]:
Importance_agg.sort_values('Importance',ascending=False).head(10)

,Importance
"(rain_mm_0, A729 VOTUPORANGA)",5.095039
"(rain_mm_0, A708 FRANCA)",2.406132
"(rain_mm_0, A706 CAMPOS DO JORDAO)",2.131853
"(rain_mm_0, A726 PIRACICABA)",1.959240
"(rain_mm_0, A727 LINS)",1.922486
"(rain_mm_7, A729 VOTUPORANGA)",1.697760
"(rain_mm_0, A737 IBITINGA)",1.673203
"(rain_mm_0, A728 TAUBATE)",1.628817
"(rain_mm_0, A711 SAO CARLOS)",1.594796
"(rain_mm_0, A740 SAO LUIS DO PARAITINGA)",1.575475


In [15]:
Importance_agg.sort_values('Importance',ascending=False).tail(10)

,Importance
"(rain_mm_14, A767 SAO SEBASTIAO)",0.001818
"(rain_mm_0, A744 BRAGANCA PAULISTA)",0.000049
"(rain_mm_14, A770 SAO SIMAO)",0.000000
"(rain_mm_7, A770 SAO SIMAO)",0.000000
"(rain_mm_30, A770 SAO SIMAO)",0.000000
"(rain_mm_14, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_0, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_0, A770 SAO SIMAO)",0.000000
"(rain_mm_30, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_7, A771 SAO PAULO - INTERLAGOS)",0.000000


## All stations but only lagged 7 days

In [17]:
#Rain lagged 7 days predicting energy generated at B. Bonita
X_7 = rain_7.drop('influent_flow', axis = 1)
y_7 = rain_7.influent_flow
X_7_train, X_7_test, y_7_train, y_7_test = train_test_split(X_7, y_7, train_size = 0.8, random_state = 0)


random_forest_rain_7 = RandomForestRegressor(max_features = 42, random_state = 1)
random_forest_rain_7.fit(X_7_train, y_7_train)
pred_7 = random_forest_rain_7.predict(X_7_test)

Importance_7 = pd.DataFrame({'Importance':random_forest_rain_7.feature_importances_*100}, 
                          index = X_7.columns)

Importance_7 = Importance_7.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_7_test, pred_7))

random forest MAE: 227.10834472512687


In [18]:
Importance_7.sort_values('Importance', ascending=False).head(10)

,Importance
"(rain_mm_7, A729 VOTUPORANGA)",9.803277
"(rain_mm_7, A708 FRANCA)",5.896072
"(rain_mm_7, A746 BARRA DO TURVO)",5.692830
"(rain_mm_7, A740 SAO LUIS DO PARAITINGA)",5.218907
"(rain_mm_7, A706 CAMPOS DO JORDAO)",5.018756
"(rain_mm_7, A712 IGUAPE)",4.840435
"(rain_mm_7, A701 SAO PAULO - MIRANTE)",3.646024
"(rain_mm_7, A715 SAO MIGUEL ARCANJO)",3.332790
"(rain_mm_7, A755 BARUERI)",3.065182
"(rain_mm_7, A718 RANCHARIA)",3.019567


In [19]:
Importance_7.sort_values('Importance', ascending=False).tail(10)

,Importance
"(rain_mm_7, A765 BERTIOGA)",0.477751
"(rain_mm_7, A762 DRACENA)",0.469475
"(rain_mm_7, A764 BEBDOURO)",0.117484
"(rain_mm_7, A768 TUPA)",0.020969
"(rain_mm_7, A763 MARILIA)",0.019571
"(rain_mm_7, A769 CACHOEIRA PAULISTA)",0.007504
"(rain_mm_7, A767 SAO SEBASTIAO)",0.005647
"(rain_mm_7, A744 BRAGANCA PAULISTA)",0.002269
"(rain_mm_7, A770 SAO SIMAO)",0.000000
"(rain_mm_7, A771 SAO PAULO - INTERLAGOS)",0.000000


## All stations lagged 14 days

In [23]:
#Rain lagged 14 days predicting energy generated at B. Bonita

X_14 = rain_14.drop('influent_flow', axis = 1)
y_14 = rain_14.influent_flow
X_14_train, X_14_test, y_14_train, y_14_test = train_test_split(X_14, y_14, train_size = 0.8, random_state = 0)

random_forest_rain_14 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_14.fit(X_14_train, y_14_train)

pred_14 = random_forest_rain_14.predict(X_14_test)

Importance_14 = pd.DataFrame({'Importance':random_forest_rain_14.feature_importances_*100}, 
                          index = X_14.columns)

Importance_14 = Importance_14.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_14_test, pred_14))


random forest MAE: 217.7499808232889


In [25]:
Importance_14.sort_values('Importance',ascending=False).head(10)

,Importance
"(rain_mm_14, A729 VOTUPORANGA)",5.995720
"(rain_mm_14, A746 BARRA DO TURVO)",4.892334
"(rain_mm_14, A708 FRANCA)",4.820513
"(rain_mm_14, A712 IGUAPE)",4.653864
"(rain_mm_14, A714 ITAPEVA)",4.222909
"(rain_mm_14, A706 CAMPOS DO JORDAO)",3.932453
"(rain_mm_14, A701 SAO PAULO - MIRANTE)",3.903729
"(rain_mm_14, A718 RANCHARIA)",3.891657
"(rain_mm_14, A740 SAO LUIS DO PARAITINGA)",3.784053
"(rain_mm_14, A715 SAO MIGUEL ARCANJO)",3.535319


In [26]:
Importance_14.sort_values('Importance',ascending=False).tail(10)

,Importance
"(rain_mm_14, A766 REGISTRO)",0.401039
"(rain_mm_14, A764 BEBDOURO)",0.195455
"(rain_mm_14, A762 DRACENA)",0.162783
"(rain_mm_14, A763 MARILIA)",0.062132
"(rain_mm_14, A768 TUPA)",0.044965
"(rain_mm_14, A767 SAO SEBASTIAO)",0.018572
"(rain_mm_14, A769 CACHOEIRA PAULISTA)",0.017569
"(rain_mm_14, A744 BRAGANCA PAULISTA)",0.004421
"(rain_mm_14, A770 SAO SIMAO)",0.000000
"(rain_mm_14, A771 SAO PAULO - INTERLAGOS)",0.000000


## All stations lagged 30 days

In [28]:
X_30 = rain_30.drop('influent_flow', axis = 1)
y_30 = rain_30.influent_flow
X_30_train, X_30_test, y_30_train, y_30_test = train_test_split(X_30, y_30, train_size = 0.8, random_state = 0)

random_forest_rain_30 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_30.fit(X_30_train, y_30_train)

pred_30 = random_forest_rain_30.predict(X_30_test)

Importance_30 = pd.DataFrame({'Importance':random_forest_rain_30.feature_importances_*100}, 
                          index = X_30.columns)

Importance_30 = Importance_30.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_30_test, pred_30))


random forest MAE: 232.57995188712061


In [29]:
Importance.sort_values('Importance',ascending=False).head(10)


NameError: name 'Importance' is not defined

In [30]:
Importance.sort_values('Importance',ascending=False).tail(10)

NameError: name 'Importance' is not defined